In [1]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install torch torchvision matplotlib tqdm



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import itertools
import os
import matplotlib.pyplot as plt

# Define the Generator and Discriminator models
class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super(ResnetBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim)
        )

    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, num_resnet_blocks=6):
        super(Generator, self).__init__()
        model = [nn.Conv2d(input_nc, 64, kernel_size=7, padding=3, bias=False),
                 nn.InstanceNorm2d(64),
                 nn.ReLU(True)]

        # Downsample
        model += [nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
                  nn.InstanceNorm2d(128),
                  nn.ReLU(True)]

        model += [nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False),
                  nn.InstanceNorm2d(256),
                  nn.ReLU(True)]

        # Resnet blocks
        for _ in range(num_resnet_blocks):
            model += [ResnetBlock(256)]

        # Upsample
        model += [nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
                  nn.InstanceNorm2d(128),
                  nn.ReLU(True)]

        model += [nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1, bias=False),
                  nn.InstanceNorm2d(64),
                  nn.ReLU(True)]

        model += [nn.Conv2d(64, output_nc, kernel_size=7, padding=3)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()
        model = [nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
                 nn.LeakyReLU(0.2, True)]

        model += [nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
                  nn.InstanceNorm2d(128),
                  nn.LeakyReLU(0.2, True)]

        model += [nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
                  nn.InstanceNorm2d(256),
                  nn.LeakyReLU(0.2, True)]

        model += [nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
                  nn.InstanceNorm2d(512),
                  nn.LeakyReLU(0.2, True)]

        model += [nn.Conv2d(512, 1, kernel_size=4, padding=1)]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


In [4]:
def get_dataloaders(data_dir, image_size=256, batch_size=1):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainA = datasets.ImageFolder(os.path.join(data_dir, 'trainA'), transform=transform)
    trainB = datasets.ImageFolder(os.path.join(data_dir, 'trainB'), transform=transform)
    testA = datasets.ImageFolder(os.path.join(data_dir, 'testA'), transform=transform)
    testB = datasets.ImageFolder(os.path.join(data_dir, 'testB'), transform=transform)

    train_loaderA = DataLoader(trainA, batch_size=batch_size, shuffle=True)
    train_loaderB = DataLoader(trainB, batch_size=batch_size, shuffle=True)
    test_loaderA = DataLoader(testA, batch_size=batch_size, shuffle=False)
    test_loaderB = DataLoader(testB, batch_size=batch_size, shuffle=False)

    return train_loaderA, train_loaderB, test_loaderA, test_loaderB


In [5]:
def train_cycleGAN(genA2B, genB2A, discA, discB, train_loaderA, train_loaderB, device, num_epochs=200, lr=0.0002):
    criterion = nn.MSELoss()
    optimizer_G = optim.Adam(itertools.chain(genA2B.parameters(), genB2A.parameters()), lr=lr, betas=(0.5, 0.999))
    optimizer_DA = optim.Adam(discA.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_DB = optim.Adam(discB.parameters(), lr=lr, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for i, (dataA, dataB) in enumerate(zip(train_loaderA, train_loaderB)):
            real_A = dataA[0].to(device)
            real_B = dataB[0].to(device)

            # Update Generators
            optimizer_G.zero_grad()

            fake_B = genA2B(real_A)
            fake_A = genB2A(real_B)

            # Cycle Consistency
            cycle_A = genB2A(fake_B)
            cycle_B = genA2B(fake_A)

            loss_cycle_A = criterion(cycle_A, real_A)
            loss_cycle_B = criterion(cycle_B, real_B)

            loss_G = loss_cycle_A + loss_cycle_B
            loss_G.backward()
            optimizer_G.step()

            # Update Discriminators
            optimizer_DA.zero_grad()
            optimizer_DB.zero_grad()

            pred_real_A = discA(real_A)
            pred_real_B = discB(real_B)

            loss_DA = criterion(pred_real_A, torch.ones_like(pred_real_A))
            loss_DB = criterion(pred_real_B, torch.ones_like(pred_real_B))

            loss_DA.backward()
            loss_DB.backward()

            optimizer_DA.step()
            optimizer_DB.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss_G: {loss_G.item()}, Loss_DA: {loss_DA.item()}, Loss_DB: {loss_DB.item()}')


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize models
genA2B = Generator(3, 3).to(device)
genB2A = Generator(3, 3).to(device)
discA = Discriminator(3).to(device)
discB = Discriminator(3).to(device)

# Load Data
data_dir = "C:/Users/shalu/Downloads/Dataset"
train_loaderA, train_loaderB, test_loaderA, test_loaderB = get_dataloaders(data_dir)

# Train the CycleGAN
train_cycleGAN(genA2B, genB2A, discA, discB, train_loaderA, train_loaderB, device, num_epochs=100)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/shalu/Downloads/Dataset/trainA'